In [1]:
import cv2 
import mediapipe as mp
from math import hypot
from ctypes import cast,POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities,IAudioEndpointVolume
import numpy as np

In [ ]:
cap=cv2.VideoCapture(0)

mpHands=mp.solutions.hands
hands=mpHands.Hands()
mpDraw=mp.solutions.drawing_utils

devices=AudioUtilities.GetSpeakers()
interface=devices.Activate(IAudioEndpointVolume._iid_,CLSCTX_ALL,None)
volume=cast(interface,POINTER(IAudioEndpointVolume))

volmin,volmax=volume.GetVolumeRange()[:2]


while True:
    success,frame=cap.read()
    frame=cv2.flip(frame,1)
    framergb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    result=hands.process(framergb)


    lmlist=[]
    if result.multi_hand_landmarks:
        for handlandmark in result.multi_hand_landmarks:
            for id,lm in enumerate(handlandmark.landmark):
                h,w,color=frame.shape
                cx,cy=int(lm.x*w),int(lm.y*h)
                lmlist.append([id,cx,cy])
            mpDraw.draw_landmarks(frame,handlandmark,mpHands.HAND_CONNECTIONS)

    if lmlist!=[]:
        x1,y1=lmlist[4][1],lmlist[4][2]
        x2,y2=lmlist[8][1],lmlist[8][2]

        cv2.circle(frame,(x1,y1),7,(255,0,0),cv2.FILLED)
        cv2.circle(frame,(x2,y2),7,(255,0,0),cv2.FILLED)
        cv2.line(frame,(x1,y1),(x2,y2),(0,255,0),3)

        length=hypot(x2-x1,y2-y1)

        vol=np.interp(length,[15,220],[volmin,volmax])
        #print(vol,length)
        volume.SetMasterVolumeLevel(vol,None)

    
    cv2.imshow('image',frame)
    if cv2.waitKey(1)& 0xff==ord('q'):
        break


: 